<a href="https://colab.research.google.com/github/matlogica/AADC-Python/blob/main/getting-started/05-scipy-interop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AADC Example how to use recorded objective functions with numerical optimizations methods from SciPy

This notebook shows how to use AADC to achieve a 30x speedup for SABR model calibration across a volatility cube
We start with a traditional approach. The calibration objective function is just a vector of calibration errors for a vector of strikes, given `sig0`, `rho`, and `nu`
We then use SciPy's least_squares method to calibrate the 3 parameters for 1000 scenarios (representing a hypothetical vol cube) using 7 strikes for each calibration.
We then proceed to use AADC to record the calibration objective function. Note, that thanks to using AVX2 we are able not only to calculate the vector of errors, but at the same time (and with the same computation effort) also a full jacobian of the objective function: the finite difference numerical derivatives of the errors wrt to the 3 parameters

In [1]:
import warnings

import numpy as np

def sabr_normal_vol_atm(fwd, expiry, beta, sig0, rho, nu):
    F = fwd
    t = expiry
    t = np.where(abs(t) < 1e-10, 1e-10, t)

    c1 = 1 + ((2 - 3 * rho ** 2) / 24) * (nu ** 2) * t
    c2 = (rho * beta * nu * t) / (4 * F ** (1 - beta))
    c3 = beta * (beta - 2) * t / (24 * F ** (2 - 2 * beta))

    sig_n = (c1 * sig0 + c2 * sig0 ** 2 + c3 * sig0 ** 3) / (F ** (-beta))

    return sig_n


def sabr_normal_vol_otm(fwd, strike, expiry, beta, sig0, rho, nu):
    F = fwd
    K = strike
    t = expiry
    t = np.where(abs(t) < 1e-10, 1e-10, t)

    k = K / F
    alpha = sig0 / (F ** (1 - beta))

    beta_close_to_one = np.isclose(beta, 1, 1e-10)
    q = np.where(beta_close_to_one, np.log(k), (k ** (1 - beta) - 1) / (1 - beta))

    z = q * nu / alpha
    z_close_to_zero = np.isclose(z, 0, 1e-10)
    z = np.where(z_close_to_zero, np.nan, z)

    _H = z / np.log((np.sqrt(1 + 2 * rho * z + z ** 2) + z + rho) / (1 + rho))

    H = np.where(z_close_to_zero, 1, _H)

    _B = np.log((q * k ** (beta / 2)) / (k - 1)) * (alpha ** 2) / (q ** 2)
    _B += (rho / 4) * ((k ** beta - 1) / (k - 1)) * alpha * nu
    _B += ((2 - 3 * rho ** 2) / 24) * (nu ** 2)

    B = ((k - 1) / q) * (1 + _B * t)

    sig_n = sig0 * (F ** beta) * H * B

    return sig_n


def sabr_normal_vol(fwd, strike, expiry, beta, sig0, rho, nu):
    F, K, expiry, beta, sig0, rho, nu = np.broadcast_arrays(fwd, strike, expiry, beta, sig0, rho, nu)

    return np.where(np.isclose(F, K, 1e-6),
                     sabr_normal_vol_atm(F, expiry, beta, sig0, rho, nu),
                     sabr_normal_vol_otm(F, K, expiry, beta, sig0, rho, nu))


Here we generate parameters for our 1000 scenarios

In [2]:
warnings.filterwarnings('ignore')
ncalibrations = 1000

# Generate market data
fwd = np.linspace(0.01, 0.05, ncalibrations)
beta = np.tile([0.5, 0.6, 0.7, 0.8, 0.9], -(-ncalibrations // 5))[:ncalibrations]
expiry = np.tile(np.linspace(1, 10, 10), -(-ncalibrations // 10))[:ncalibrations]

np.random.seed(42)
sig0 = np.random.uniform(0.01, 0.05, ncalibrations)
rho = np.random.uniform(-0.5, 0., ncalibrations)
nu = np.random.uniform(0.2, 0.6, ncalibrations)

# 7 strikes, each row is an independent calibration scenario
strikes = np.linspace(0.25, 1.75, 7).reshape(-1, 1) * fwd
vols = sabr_normal_vol(fwd, strikes, expiry, beta, sig0, rho, nu)
strikes,vols

(array([[0.0025    , 0.00251001, 0.00252002, ..., 0.01247998, 0.01248999,
         0.0125    ],
        [0.005     , 0.00502002, 0.00504004, ..., 0.02495996, 0.02497998,
         0.025     ],
        [0.0075    , 0.00753003, 0.00756006, ..., 0.03743994, 0.03746997,
         0.0375    ],
        ...,
        [0.0125    , 0.01255005, 0.0126001 , ..., 0.0623999 , 0.06244995,
         0.0625    ],
        [0.015     , 0.01506006, 0.01512012, ..., 0.07487988, 0.07493994,
         0.075     ],
        [0.0175    , 0.01757007, 0.01764014, ..., 0.08735986, 0.08742993,
         0.0875    ]]),
 array([[0.00250636, 0.00262104, 0.00220094, ..., 0.00478576, 0.00637934,
         0.00728143],
        [0.00252886, 0.00278514, 0.0020041 , ..., 0.00404589, 0.00591273,
         0.0060057 ],
        [0.00251032, 0.00290853, 0.00176953, ..., 0.00306579, 0.00518306,
         0.00427347],
        ...,
        [0.00252301, 0.00320189, 0.0019685 , ..., 0.00232997, 0.00447891,
         0.00362963],
        [0.0

The `residual` function is the calibration objective. Note how we're using "walls" to constrain `rho` within `(-1,1)`

In [3]:
from scipy.optimize import least_squares
vol_weights = [1., 1., 1., 100., 1., 1., 1.]

def residual(x, fwd, beta, expiry, strikes, vols):
    sig0, rho, nu = x
    rho = np.broadcast_to(rho, vols.shape)

    return np.where(np.abs(rho) > 0.9999,
                    np.ones_like(vols) * 1e6,
                    vol_weights * (sabr_normal_vol(fwd, strikes, expiry, beta, sig0, rho, nu) - vols))

x0 = np.array([0.02, -0.25, 0.03])

def sabr_normal_smile_fit(scen):
    results = least_squares(
        residual,
        x0,
        args=(fwd[scen], beta[scen], expiry[scen], strikes[:, scen], vols[:, scen]),
        method="lm",
        xtol=1e-6)

    return results.x

Solve for sig0, rho, nu for each scenario: `sig0_bar`, `rho_bar`, `nu_bar` are calibrated parameters so far without any use of AADC. Note the cell computation time.

In [4]:
%%time
sig0_bar, rho_bar, nu_bar = np.transpose([sabr_normal_smile_fit(j) for j in range(ncalibrations)])

rho_bar, nu_bar = np.where(nu_bar < 0, -rho_bar, rho_bar), np.abs(nu_bar)

CPU times: user 9.73 s, sys: 0 ns, total: 9.73 s
Wall time: 9.78 s


In [5]:
assert np.allclose(sig0_bar, sig0, atol=1e-10)
assert np.allclose(rho_bar, rho, atol=1e-10)
assert np.allclose(nu_bar, nu, atol=1e-10)
# sig0_bar - sig0, rho_bar - rho, nu_bar - nu
# np.argmax(np.abs(sig0_bar - sig0)), np.argmax(np.abs(rho_bar - rho)), np.argmax(np.abs(nu_bar - nu))


Now AADC the whole thing. `aadc.record` returns the recorded AADC kernel for `residual` function, and has 3 members of interest:
- `kernel.func` is the AADC JIT version of `residual` and can be substituted for it in `least_squares`, except for a small detail:
- `kernel.set_params` should be used to curry the extra arguments of the objective function that are not part of the calibration
- `kernel.jac` is the numerical jacobian, and it can be fed to `least_squares` to achieve faster convergence

### Please uncomment next line if you don't have AADC installed locally

In [6]:
import sys
#!pip install https://matlogica.com/DemoReleases/aadc-1.7.5.30-cp3{sys.version_info.minor}-cp3{sys.version_info.minor}-linux_x86_64.whl

In [7]:
import aadc
warnings.filterwarnings('ignore')

kernel = aadc.record(residual, x0, params=(fwd[0], beta[0], expiry[0], strikes[:, 0], vols[:, 0]), bump_size=1e-10)

You are using evaluation version of AADC. Expire date is 20240901


In [8]:
def sabr_normal_smile_fit_aadc(scen):
    kernel.set_params(fwd[scen], beta[scen], expiry[scen], strikes[:, scen], vols[:, scen])
    results = least_squares(
        kernel.func,
        x0,
        jac=kernel.jac,
        method="lm",
        xtol=1e-6)

    return results.x

Moment of truth. Let's check the solution we obtain with the JIT kernel is the same

In [9]:
%%time
sig0_star, rho_star, nu_star = np.transpose([sabr_normal_smile_fit_aadc(j) for j in range(ncalibrations)])
rho_star, nu_star = np.where(nu_star < 0, -rho_star, rho_star), np.abs(nu_star)

assert np.allclose(sig0_bar, sig0_star, atol=1e-10)
assert np.allclose(rho_bar, rho_star, atol=1e-10)
assert np.allclose(nu_bar, nu_star, atol=1e-10)

# sig0_star - sig0_bar, rho_star - rho_bar, nu_star - nu_bar
# np.argmax(np.abs(sig0_star - sig0_bar)), np.argmax(np.abs(rho_star - rho_bar)), np.argmax(np.abs(nu_star - nu_bar))



CPU times: user 340 ms, sys: 3.03 ms, total: 343 ms
Wall time: 342 ms
